In [1]:
import torch
print(torch.cuda.is_available())

True


In [15]:
import cv2

# Open the webcam (0 is usually the default webcam, you can change it if needed)
cap = cv2.VideoCapture(5)  # Ensure V4L2 backend is used

# Check if the camera opened successfully
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Capture a single frame
ret, frame = cap.read()

# Check if frame is captured
if not ret:
    print("Error: Failed to capture image.")
else:
    # Save the captured image
    cv2.imwrite("captured_image.jpg", frame)
    print("Image saved as captured_image.jpg")
    print(frame.shape)
    cropped_image = frame[:, 75:555]  # Example: Crop a 200x200 region
    cv2.imwrite("cropped_image.jpg", cropped_image)
    print("Cropped image saved as cropped_image.jpg")
    print(cropped_image.shape)
    resized_image = cv2.resize(cropped_image, (256, 256))
    cv2.imwrite("resized_image.jpg", resized_image)
    print("Resized image saved as resized_image.jpg")
    print(resized_image.shape)
    flipped_image = cv2.flip(resized_image, 0)
    cv2.imwrite("flipped_image.jpg", flipped_image)
    print("Flipped image saved as flipped_image.jpg")


# Release the webcam and close any OpenCV windows
cap.release()
cv2.destroyAllWindows()


Image saved as captured_image.jpg
(480, 640, 3)
Cropped image saved as cropped_image.jpg
(480, 480, 3)
Resized image saved as resized_image.jpg
(256, 256, 3)
Flipped image saved as flipped_image.jpg


In [2]:
import pyspacemouse
import time

In [ ]:
success = pyspacemouse.open(dof_callback=pyspacemouse.print_state, button_callback=pyspacemouse.print_buttons)
if success:
    while 1:
        state = pyspacemouse.read()
        time.sleep(1)
        print(state)

In [22]:
from time import sleep
import sys
sys.path.append('./dynamixel_controller_main')

# Import from parent directory
from dynamixel_controller import Dynamixel

servo = Dynamixel(ID=10, descriptive_device_name="XM430 test motor", 
                    series_name="xm", baudrate=57600, port_name="/dev/ttyUSB0")



def button_0(state, buttons, pressed_buttons):
    print("Button:", pressed_buttons)


def button_0_1(state, buttons, pressed_buttons):
    print("Buttons:", pressed_buttons)


def someButton(state, buttons):
    #print("Some button")
    pass


def callback():
    button_arr = [pyspacemouse.ButtonCallback(0, button_0),
                  pyspacemouse.ButtonCallback([1], lambda state, buttons, pressed_buttons: print("Button: 1")),
                  pyspacemouse.ButtonCallback([0, 1], button_0_1), ]

    success = pyspacemouse.open(dof_callback=pyspacemouse.print_state, button_callback=someButton,
                                button_callback_arr=button_arr)
    if success:
        while True:
            state = pyspacemouse.read()
            if state.buttons[1] == 1:

                servo.begin_communication()
                servo.set_operating_mode("velocity")

                for i in range(1):
                    servo.write_velocity(100)
                    sleep(0.5)

                    servo.write_velocity(-100)
                    sleep(0.5)

                servo.end_communication()



callback()

SpaceMouse Compact found
SpaceMouse Compact found
SpaceMouse Compact found
Button: 1
Port open successfully for: XM430 test motor
Baudrate set successfully for: XM430 test motor
Torque enable successful for: XM430 test motor ID: 10
Mode set to velocity control successful for: XM430 test motor ID: 10
Torque disable successful for: XM430 test motor ID: 10
Port closed successfully for: XM430 test motor
Button: 1
Port open successfully for: XM430 test motor
Baudrate set successfully for: XM430 test motor
Torque enable successful for: XM430 test motor ID: 10
Mode set to velocity control successful for: XM430 test motor ID: 10
Torque disable successful for: XM430 test motor ID: 10
Port closed successfully for: XM430 test motor
Button: 1
Port open successfully for: XM430 test motor
Baudrate set successfully for: XM430 test motor
Torque enable successful for: XM430 test motor ID: 10
Mode set to velocity control successful for: XM430 test motor ID: 10
Torque disable successful for: XM430 test m

KeyboardInterrupt: 

In [5]:
from time import sleep
import sys
sys.path.append('./dynamixel_controller_main')

# Import from parent directory
from dynamixel_controller import Dynamixel


initial_pwm = 200       # Starting PWM value (small positive)
max_pwm = 600           # Max PWM to prevent over-force (adjust based on your gripper)
current_threshold = 250  # mA, threshold to detect object gripping
def gripper_grab(servo):
    servo.torque_off()
    servo.set_operating_mode("pwm")  # Switch to PWM control mode
    servo.torque_on()

    print(f"Gripper is closing with increasing PWM...")

    pwm_value = initial_pwm  # Start with low PWM value
    while pwm_value < max_pwm:
        sleep(0.1)  # Wait a bit before checking

        actual_current = servo.read_current()  # XL430 still allows reading actual current
        print(f"Actual Current: {actual_current} mA")

        # Check if gripping (i.e., resistance detected)
        if actual_current >= current_threshold:
            print(f"Gripper has tightened. Stopping motor.")
            servo.write_pwm(0)  # Stop applying force
            break

        # Increase the PWM gradually to apply more gripping force
        pwm_value += 10  # Increment PWM slowly
        if pwm_value > max_pwm:
            pwm_value = max_pwm

        servo.write_pwm(pwm_value)

    print("Gripper grabbing finished.")

servo = Dynamixel(ID=1, descriptive_device_name="XM430 test motor", 
                    series_name="xm", baudrate=57600, port_name="/dev/ttyUSB0")


servo.begin_communication()
servo.set_operating_mode("velocity")

for i in range(1):
    servo.write_velocity(100)
    sleep(0.5)

    servo.write_velocity(-100)
    sleep(0.5)


gripper_grab(servo)

servo.end_communication()

Port open successfully for: XM430 test motor
Baudrate set successfully for: XM430 test motor
Torque enable successful for: XM430 test motor ID: 1
Mode set to velocity control successful for: XM430 test motor ID: 1
Mode set to pwm control successful for: XM430 test motor ID: 1
Gripper is closing with increasing PWM...
Actual Current: 93 mA
Actual Current: 119 mA
Actual Current: 76 mA
Actual Current: 66 mA
Actual Current: 58 mA
Actual Current: 57 mA
Actual Current: 52 mA
Actual Current: 42 mA
Actual Current: 37 mA
Actual Current: 39 mA
Actual Current: 40 mA
Actual Current: 39 mA
Actual Current: 42 mA
Actual Current: 40 mA
Actual Current: 42 mA
Actual Current: 70 mA
Actual Current: 310 mA
Gripper has tightened. Stopping motor.
Gripper grabbing finished.
Torque disable successful for: XM430 test motor ID: 1
Port closed successfully for: XM430 test motor


In [24]:
import rtde_receive
import rtde_control

In [2]:
import cv2

# Open the webcam (0 is usually the default webcam, you can change it if needed)
cap = cv2.VideoCapture(4)  # Ensure V4L2 backend is used

# Check if the camera opened successfully
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Capture a single frame
ret, frame = cap.read()

# Check if frame is captured
if not ret:
    print("Error: Failed to capture image.")
else:
    # Save the captured image
    cv2.imwrite("captured_image.jpg", frame)
    print("Image saved as captured_image.jpg")
    print(frame.shape)
    cropped_image = frame[:, 50:530]  # Example: Crop a 200x200 region
    cv2.imwrite("cropped_image.jpg", cropped_image)
    print("Cropped image saved as cropped_image.jpg")
    print(cropped_image.shape)
    resized_image = cv2.resize(cropped_image, (256, 256), interpolation=cv2.INTER_AREA)
    cv2.imwrite("resized_image.jpg", resized_image)
    print("Resized image saved as resized_image.jpg")
    print(resized_image.shape)

# Release the webcam and close any OpenCV windows
cap.release()
cv2.destroyAllWindows()



Image saved as captured_image.jpg
(480, 640, 3)
Cropped image saved as cropped_image.jpg
(480, 480, 3)
Resized image saved as resized_image.jpg
(256, 256, 3)


In [2]:
import h5py

# Open the HDF5 file in read mode
with h5py.File("libero_spatial/pick_up_the_black_bowl_next_to_the_cookie_box_and_place_it_on_the_plate_demo.hdf5", "r") as f:
    # List all groups and datasets inside the file
    print("Keys in HDF5 file:", list(f.keys()))
    
    # Get the dataset
    group = f["data"]  # Access the "data" group
    # List all datasets or sub-groups inside "data"
    print("Members of 'data':", list(group.keys()))
    demo_0 = group["demo_0"]  # Access the "demo_0" dataset
    print("Keys of 'demo_0':", demo_0.keys())
    actions = demo_0["actions"]  # Access the "actions" dataset
    print("Actions shape:", actions.shape)
    dones = demo_0["dones"]  # Access the "dones" dataset
    print("Dones shape:", dones.shape)
    obs = demo_0["obs"]  # Access the "obs" dataset
    print("Observations:", obs.keys())
    states = obs["ee_states"]  # Access the "states" dataset
    print("States shape:", states.shape)
    print(states[0])
    
    gripper_states = obs["gripper_states"]  # Access the "gripper_states" dataset
    print("Gripper states shape:", gripper_states.shape)
    print(gripper_states[0])
    joint_states = obs["joint_states"]  # Access the "joint_states" dataset
    print("Joint states shape:", joint_states.shape)
    print(joint_states[0])
    

    #for i in range(5):
        #print(i+50)
        #print("actions:", actions[i+50])
        #print("states:", states[i+50])
        #print("joint_states:", joint_states[i+50])
    print(demo_0["robot_states"].shape)
    print(demo_0["states"].shape)
# Open the HDF5 file in read mode
with h5py.File("libero_spatial/pick_up_the_black_bowl_next_to_the_cookie_box_and_place_it_on_the_plate_demo.hdf5", "r") as f:
    # List all groups and datasets inside the file
    print("Keys in HDF5 file:", list(f.keys()))
    
    # Get the dataset
    group = f["data"]  # Access the "data" group
    # List all datasets or sub-groups inside "data"
    print("Members of 'data':", list(group.keys()))
    demo_0 = group["demo_1"]  # Access the "demo_0" dataset
    print("Keys of 'demo_0':", demo_0.keys())
    actions = demo_0["actions"]  # Access the "actions" dataset
    print("Actions shape:", actions.shape)
    dones = demo_0["dones"]  # Access the "dones" dataset
    print("Dones shape:", dones.shape)
    obs = demo_0["obs"]  # Access the "obs" dataset
    print("Observations:", obs.keys())
    states = obs["ee_states"]  # Access the "states" dataset
    print("States shape:", states.shape)
    print(states[0])
    
    gripper_states = obs["gripper_states"]  # Access the "gripper_states" dataset
    print("Gripper states shape:", gripper_states.shape)
    print(gripper_states[0])
    joint_states = obs["joint_states"]  # Access the "joint_states" dataset
    print("Joint states shape:", joint_states.shape)
    print(joint_states[0])
    

    #for i in range(5):
        #print(i+50)
        #print("actions:", actions[i+50])
        #print("states:", states[i+50])
        #print("joint_states:", joint_states[i+50])
    print(demo_0["robot_states"].shape)
    print(demo_0["states"].shape)

Keys in HDF5 file: ['data']
Members of 'data': ['demo_0', 'demo_1', 'demo_10', 'demo_11', 'demo_12', 'demo_13', 'demo_14', 'demo_15', 'demo_16', 'demo_17', 'demo_18', 'demo_19', 'demo_2', 'demo_20', 'demo_21', 'demo_22', 'demo_23', 'demo_24', 'demo_25', 'demo_26', 'demo_27', 'demo_28', 'demo_29', 'demo_3', 'demo_30', 'demo_31', 'demo_32', 'demo_33', 'demo_34', 'demo_35', 'demo_36', 'demo_37', 'demo_38', 'demo_39', 'demo_4', 'demo_40', 'demo_41', 'demo_42', 'demo_43', 'demo_44', 'demo_45', 'demo_46', 'demo_47', 'demo_48', 'demo_49', 'demo_5', 'demo_6', 'demo_7', 'demo_8', 'demo_9']
Keys of 'demo_0': <KeysViewHDF5 ['actions', 'dones', 'obs', 'rewards', 'robot_states', 'states']>
Actions shape: (123, 7)
Dones shape: (123,)
Observations: <KeysViewHDF5 ['agentview_rgb', 'ee_ori', 'ee_pos', 'ee_states', 'eye_in_hand_rgb', 'gripper_states', 'joint_states']>
States shape: (123, 6)
[-0.21485152 -0.00321868  1.15933467  3.14325605  0.02287673 -0.13038479]
Gripper states shape: (123, 2)
[ 0.03623

In [ ]:
"""

Things need to record:

In each demo
Observation:
    1. image (256*256*3, jpeg)
    2. wrist image (256*256*3, jpeg)
    3. state(6D pose + 2D gripper)
    4. joint state(6 joints + 1 timestamp)
Action:
    5. 

"""

In [19]:


import cv2

cap = cv2.VideoCapture(2)  # Change index if using a different camera

# Disable auto exposure
#cap.set(cv2.CAP_PROP_AUTO_EXPOSURE, 0.25)  # 0.25 for manual mode (might vary)

# Set exposure value (experiment with values, usually -5 to 5 or a specific range)
#cap.set(cv2.CAP_PROP_EXPOSURE, 0)  # Adjust value as needed

# Check if exposure is set correctly
print("Exposure:", cap.get(cv2.CAP_PROP_EXPOSURE))

ret, frame = cap.read()

frame = frame[:, 75:555]   
cv2.imwrite("Camera_test1.png", frame)  # Save the image
    
print(frame.shape)
resized_image_3rd = cv2.resize(frame, (256, 256), interpolation=cv2.INTER_AREA)
cv2.imwrite("resized_image_3rd.jpg", resized_image_3rd)
cap.release()
cv2.destroyAllWindows()

Exposure: 50.0
(480, 480, 3)


In [2]:
import cv2

image_wrist = cv2.imread('/home/create/Documents/Haochen/data_collection/demo_0/images_wrist/resized/captured_image_wrist_1.jpg')
#image_wrist_rgb = cv2.cvtColor(image_wrist, cv2.COLOR_BGR2RGB)
print(image_wrist.shape)

(256, 256, 3)


In [5]:
import numpy as np

data = np.load("/home/create/Documents/Haochen/data/val/demo_109.npy", allow_pickle=True)
print(data)


[{'image': array([[[176, 172, 163],
         [176, 172, 163],
         [177, 173, 164],
         ...,
         [204, 193, 189],
         [207, 192, 187],
         [208, 191, 184]],

        [[177, 173, 164],
         [177, 173, 164],
         [177, 173, 164],
         ...,
         [203, 193, 191],
         [205, 194, 190],
         [206, 193, 187]],

        [[177, 173, 164],
         [177, 173, 164],
         [177, 173, 164],
         ...,
         [200, 195, 192],
         [198, 193, 190],
         [198, 193, 190]],

        ...,

        [[195, 184, 166],
         [195, 184, 166],
         [195, 184, 166],
         ...,
         [227, 210, 192],
         [227, 210, 192],
         [227, 210, 192]],

        [[195, 184, 166],
         [195, 184, 166],
         [195, 184, 166],
         ...,
         [227, 210, 192],
         [227, 210, 192],
         [227, 210, 192]],

        [[195, 184, 166],
         [195, 184, 166],
         [195, 184, 166],
         ...,
         [227, 210, 192]